# Homework (40 pts) - PCA and Classification

---
## Load EEG recordings...

### `eeg_uV[channel, time point, trial]`

In [3]:
from scipy.io import loadmat

data = loadmat('eeg.mat')

# grab relevant data, EEG units are microvolts (uV)
# each channel is an electrode, each trial is a separate EEG recording from that electrode
eeg_uV = data["EEG"][0,0]["data"]  # [channel, time, trial]
time_ms = data["EEG"][0,0]["times"][0]

eeg_uV.shape, time_ms.shape

((64, 640, 99), (640,))

---
1. (5 pts) From the EEG data above, generate the average EEG across trials for each channel for times between 0 and 500 ms. Plot the result for channel 20.

---
2. (5 pts) Apply PCA to the averaged EEGs from #1 above. Use as many principal components (PCs) as needed to explain 90% of the variance in the EEG waveforms. Report the number of PCs used and plot each PC as if it were an EEG waveform itself.

---
3. (5 pts) Replot the average EEG for channel 20 as in #1 above, and additionally overlay the plot of its projection onto the principal components from #2 above. Use a legend to label the plots as 'original' and 'projected'. Does it look like we lost a lot of information by describing each waveform using only a small number of PCs as opposed to 129 time points?

---
4. (5 pts) How much of each of the PCs were mixed together to generate the projection for channel 20 as shown in #3 above? Plot the relative amounts of each PC as a bar graph. Use only a single tick mark on the x-axis for each PC.

---
5. (5 pts) Use k-means to group the EEG waveforms into three clusters in the PCA reduced dimensions. Plot the EEG waveforms as points in the PCA reduced dimensions (axes are the PCs) colored according to their cluster. For visualization, if you have more than three PCs, use only the first three PCs in the plot.

---
6. (5 pts) For each of the three clusters from #5 above, plot all of the averaged EEGs belonging to that cluster overlaid on each other. Additionally, overlay on each cluster plot the cluster average plotted as a black line of width 2. How well do you think the clustering did?

---
7. (5 pts) Repeat #5-#6, but this time use a Gaussian mixture model to cluster the EEGs. Do you prefer the clusters generated by k-means of GMM?

---
8. (5 pts) Repeat #5-#6, but this time use heirarchical agglomerative clustering with euclidian affinity and ward linkage.